## Google earth engine download

generally followed setup here: 

https://courses.spatialthoughts.com/install-gee-python-api.html

In [1]:
import ee
import geemap

In [2]:
ee.Authenticate()

True

In [3]:
ee.Initialize(project='ee-cehanagan')

## Sentinel 2 Band 4

In [ ]:
import ee
import geopandas as gpd

# Initialize Earth Engine
ee.Initialize()

# Load GeoJSON polygon (your AOI)
gdf = gpd.read_file("/Volumes/Myanmar 1/GIS/SentinelRoughBoundaries.geojson")

# Extract coordinates from the first feature (ensure it's a Polygon, not MultiPolygon)
polygon_coords = gdf.geometry[0].__geo_interface__['coordinates'][0]
polygon = ee.Geometry.Polygon(polygon_coords)

# Define time range
start_date = '2025-03-25'
end_date = '2025-03-28'
band = 4

# Filter Sentinel-2 collection
image = (
    ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
    .filterBounds(polygon)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 100))
    .select(f'B{band}')
    .median()
    .clip(polygon)
)

# Export parameters
task = ee.batch.Export.image.toDrive(
    image=image,
    description=f'Sentinel2_B{band}_Export',
    folder='EarthEngineExports',  # Appears in your Google Drive
    fileNamePrefix=f'sentinel2_band{band}_{start_date}_{end_date}',
    region=polygon,
    scale=10,  # Sentinel-2 Band 4 native resolution
    crs='EPSG:32647',  # Example: UTM Zone 47N (Myanmar region)
    maxPixels=1e13
)

# Start export
task.start()
print('Export started. Monitor progress at https://code.earthengine.google.com/tasks')


Export started. Monitor progress at https://code.earthengine.google.com/tasks


In [ ]:
%%bash
## Sentinel from GEE, Example of merging the results for Myanmar
gdalwarp -t_srs EPSG:32647 -tr 10 10 -srcnodata 0 -dstnodata 0 -overwrite -of GTiff -co COMPRESS=LZW -co ZLEVEL=9 -co BIGTIFF=YES \
    /Volumes/Myanmar\ 1/Imagery/Sentinel/EarthEngineExportsPost/*tif \
    /Volumes/Myanmar\ 1/Imagery/Sentinel/EarthEngineExportsPost/S2_20250330_20250401_B04_merge.tif